In [1]:
from EPA import EPA
from GJK import GJK
from Polytope import Polytope
from Simplex import Simplex
from Support import support

import numpy as np
import open3d as o3d
import itertools

In [2]:
def op3D(points, couleur = [1, 0, 0]):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(np.full((len(points), 3), couleur))
    return pcd

def visualisation(see_what : list):
    create_coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=[0, 0, 0])
    see_what.append(create_coordinate_frame)
    o3d.visualization.draw_geometries(see_what, mesh_show_wireframe=False )
     
def op3D_ligne(points):
    lines = []
    for i, j in itertools.combinations(range(len(points)), 2):
        lines.append([i, j])

    lines = np.array(lines)
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(points),
        lines=o3d.utility.Vector2iVector(lines),)
    
    return line_set

def Mesh(points,triangles):
    
    mesh = o3d.geometry.TriangleMesh(
            vertices=o3d.utility.Vector3dVector(points),
            triangles=o3d.utility.Vector3iVector(triangles),
            )
    return mesh

In [12]:
def generate_sphere(radius, center, num_points):
    points = []
    for _ in range(num_points):
        theta = 2 * np.pi * np.random.rand()
        phi = np.arccos(2 * np.random.rand() - 1)
        x = center[0] + radius * np.sin(phi) * np.cos(theta)
        y = center[1] + radius * np.sin(phi) * np.sin(theta)
        z = center[2] + radius * np.cos(phi)
        points.append([x, y, z])
    return np.array(points)
# Générer deux sphères
sphere_1 = generate_sphere(1., [0., 0., 0.], 100000)  # rayon 1, centre à l'origine, 1000 points
sphere_2 = generate_sphere(1.5, [1., 0., 0.], 1000000)  # rayon 1, centre à (2,2,2), 1000 points

sph_1 = op3D(sphere_1)
sph_2 = op3D(sphere_2,[0,0,0])

In [7]:
visualisation([sph_1,sph_2])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [13]:
direction = np.array([1,0,0]) 
simplex = [support(sphere_1, sphere_2, -direction)]
simp = Simplex(simplex)

GJK(sphere_1, sphere_2,simp)

True

In [14]:
eps = 1e-16
poly ,sup , i  = EPA(sphere_1,sphere_2,simp.simplex,eps)

In [15]:
sup

array([ 1.49993175, -0.01107407,  0.00701255])

In [16]:
i

68

In [17]:
def verif(Visu):        #vertexs,faces):
    #Visu =  mesh(np.array(vertexs),np.array(faces))
    point_cloud = Visu.sample_points_uniformly(100000)
    hull, _ = point_cloud.compute_convex_hull()
    hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
    hull_ls.paint_uniform_color((1, 0, 0))
    visualisation([hull_ls,point_cloud])
    
    #return hull_ls,point_cloud

- https://blog.winter.dev/2020/gjk-algorithm/
- https://github.com/AlexanderFabisch/distance3d/tree/master/distance3d
- http://vmlblog.free.fr/index.php?article52/algorithme-gjk
- https://www.toptal.com/game/video-game-physics-part-ii-collision-detection-for-solid-objects
- https://www.youtube.com/watch?v=DGVZYdlw_uo&t=197s

- https://blog.winter.dev/2020/epa-algorithm/
- https://handmade.network/forums/t/1521-gjk_+_expanding_polytope_algorithm_-_implementation_and_visualization
- https://github.com/SarahWeiii/CoACD?search=1

In [20]:
import trimesh 
import coacd



tool = trimesh.load('../Object/collision.stl')#'Object/Deposition Head.stl')

mesh = coacd.Mesh(tool.vertices, tool.faces)

parts = coacd.run_coacd(mesh)

full=[]
for part in parts:
    Visu= Mesh(part[0],part[1])
    point_cloud = Visu.sample_points_uniformly(10000)
    full.append(point_cloud)

    
collision = o3d.io.read_triangle_mesh('../Object/collision.stl')
visualisation(full+[collision])

[2023-05-23 17:12:23.884] [CoACD] [info] threshold               0.05
[2023-05-23 17:12:23.884] [CoACD] [info] max # convex hull       -1
[2023-05-23 17:12:23.884] [CoACD] [info] preprocess              true
[2023-05-23 17:12:23.884] [CoACD] [info] preprocess resolution   30
[2023-05-23 17:12:23.884] [CoACD] [info] pca                     false
[2023-05-23 17:12:23.884] [CoACD] [info] mcts max depth          3
[2023-05-23 17:12:23.884] [CoACD] [info] mcts nodes              20
[2023-05-23 17:12:23.884] [CoACD] [info] mcts iterations         150
[2023-05-23 17:12:23.884] [CoACD] [info] merge                   true
[2023-05-23 17:12:23.884] [CoACD] [info] seed                    1234
[2023-05-23 17:12:23.884] [CoACD] [info]  - Preprocess
[2023-05-23 17:12:23.884] [CoACD] [info] Preprocess resolution: 30
[2023-05-23 17:12:23.920] [CoACD] [info] Preprocess Time: 0.169762s
[2023-05-23 17:12:23.921] [CoACD] [info] # Points: 19164
[2023-05-23 17:12:23.921] [CoACD] [info] # Triangles: 38324
[2